In [222]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgm

from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet

from sklearn.metrics import mean_squared_error

In [223]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [224]:
sample_sub = pd.read_csv("sample_submission.csv")
train_df = pd.read_csv("train (1).csv")
test_df = pd.read_csv("test (1).csv")

In [225]:
X_train = train_df.drop(columns={'Id','Property_A','Property_B','Property_C','Property_D','Property_E','Property_F'})
y_train = train_df[['Property_A','Property_B','Property_C','Property_D','Property_E','Property_F']]

In [226]:
sensor1_X_train = X_train[['A ','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P']]

In [227]:
sensor1_X_test = test_df[['A ','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P']]


In [228]:
sensor2_X_train = X_train.drop(columns={'A ','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P'})

In [229]:
sensor2_X_test = test_df.drop(columns={'A ','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Id'})


# Predicting Individual classes 

## Predicting a first 

In [230]:
x_train_a_sensor1 = sensor1_X_train[['B','D','G','H','I','J','K','L','N']]

In [231]:
y_train_a = y_train['Property_A']

In [232]:
x_train_a_sensor2 = sensor2_X_train

In [233]:
x_train_a_together = pd.concat([x_train_a_sensor1, x_train_a_sensor2], axis=1)

In [234]:
x_test_a_sensor1 =  sensor1_X_test[['B','D','G','H','I','J','K','L','N']]

In [235]:
x_test_a_sensor2 = sensor2_X_test

In [236]:
x_test_a_together = pd.concat([x_test_a_sensor1, x_test_a_sensor2], axis=1)

In [237]:
# Normalize
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(x_train_a_together)

# transform training data
X_train_norm = norm.transform(x_train_a_together)
X_train_norm_df = pd.DataFrame(X_train_norm)

# transform testing dataabs
X_test_norm = norm.transform(x_test_a_together)
X_test_norm_df = pd.DataFrame(X_test_norm)

In [238]:
#  Standardize
from sklearn.preprocessing import StandardScaler
 
scalar = StandardScaler()
 
# fitting
scalar.fit(X_train_norm_df)
X_train_stan = scalar.transform(X_train_norm_df)
X_train_stan_df = pd.DataFrame(X_train_stan)

scalar.fit(X_test_norm_df)
X_test_stan = scalar.transform(X_test_norm_df)
X_test_stan_df = pd.DataFrame(X_test_stan)

In [239]:
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_train_stan_df, y_train_a, test_size=0.30, random_state=42)

In [240]:
# get a list of base models
def get_models():
    models = list()
    models.append(('random forest' , RandomForestRegressor(n_estimators=38,max_depth=10, random_state=0)))
    models.append(('xbg', xgb.XGBRegressor()))
    models.append(('lgm', lgm.LGBMRegressor()))
    return models


In [241]:
# evaluate each base model
def evaluate_models(models, X_train, X_val, y_train, y_val):
 # fit and evaluate the models
    scores = list()
    for name, model in models:
 # fit the model
        model.fit(X_train, y_train)
 # evaluate the model
        yhat = model.predict(X_val)
        mse = mean_squared_error(y_val, yhat)
 # store the performance
        scores.append(mse)
 # report model performance
    return scores

In [242]:
from sklearn.ensemble import VotingRegressor

models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X_train_3, X_test_3, y_train_3, y_test_3)
print(scores)
# create the ensemble
ensemble = VotingRegressor(estimators=models, weights=scores)
# fit the ensemble on the training dataset
ensemble.fit(X_train_stan_df, y_train_a)
# make predictions on test set
yhat_a = ensemble.predict(X_test_stan_df)

[0.17207977146769973, 0.22100050126939502, 0.19150369218147353]


## Predicting b 

In [244]:
x_train_b_sensor1 = sensor1_X_train[['C','D','L','M','N','O']]

In [245]:
y_train_b = y_train['Property_B']

In [246]:
x_train_b_sensor2 = sensor2_X_train

In [247]:
x_train_b_together = pd.concat([x_train_b_sensor1, x_train_b_sensor2], axis=1)

In [248]:
x_test_b_sensor1 =  sensor1_X_test[['C','D','L','M','N','O']]

In [249]:
x_test_b_sensor2 = sensor2_X_test

In [250]:
x_test_b_together = pd.concat([x_test_b_sensor1, x_test_b_sensor2], axis=1)

In [251]:
# Normalize
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(x_train_b_together)

# transform training data
X_train_norm = norm.transform(x_train_b_together)
X_train_norm_df = pd.DataFrame(X_train_norm)

# transform testing dataabs
X_test_norm = norm.transform(x_test_b_together)
X_test_norm_df = pd.DataFrame(X_test_norm)

In [252]:
#  Standardize
from sklearn.preprocessing import StandardScaler
 
scalar = StandardScaler()
 
# fitting
scalar.fit(X_train_norm_df)
X_train_stan = scalar.transform(X_train_norm_df)
X_train_stan_df = pd.DataFrame(X_train_stan)

scalar.fit(X_test_norm_df)
X_test_stan = scalar.transform(X_test_norm_df)
X_test_stan_df = pd.DataFrame(X_test_stan)

In [253]:
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_train_stan_df, y_train_b, test_size=0.30, random_state=42)

In [256]:
from sklearn.ensemble import VotingRegressor

models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X_train_3, X_test_3, y_train_3, y_test_3)
print(scores)
# create the ensemble
ensemble = VotingRegressor(estimators=models, weights=scores)
# fit the ensemble on the training dataset
ensemble.fit(X_train_stan_df, y_train_b)
# make predictions on test set
yhat_b = ensemble.predict(X_test_stan_df)

[0.7439186269813826, 0.5757829792537359, 0.5353211055927855]


## Predicting c

In [259]:
x_train_c_sensor1 = sensor1_X_train[['B','C','D','F','H','I','J','K','L','M','N']]

In [260]:
y_train_c = y_train['Property_C']

In [261]:
list_1 = sensor2_X_train.columns

list_3 = []

for i in list_1:
    if int(i[:4]) in range(550,600):
        list_3.append(i)

In [262]:
x_train_c_sensor2 = sensor2_X_train[list_3]

In [263]:
x_train_c_together = pd.concat([x_train_c_sensor1, x_train_c_sensor2], axis=1)

In [264]:
x_test_c_sensor1 =  sensor1_X_test[['B','C','D','F','H','I','J','K','L','M','N']]

In [265]:
x_test_c_sensor2 = sensor2_X_test[list_3]

In [266]:
x_test_c_together = pd.concat([x_test_c_sensor1, x_test_c_sensor2], axis=1)

In [267]:
# Normalize
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(x_train_c_together)

# transform training data
X_train_norm = norm.transform(x_train_c_together)
X_train_norm_df = pd.DataFrame(X_train_norm)

# transform testing dataabs
X_test_norm = norm.transform(x_test_c_together)
X_test_norm_df = pd.DataFrame(X_test_norm)

In [268]:
#  Standardize
from sklearn.preprocessing import StandardScaler
 
scalar = StandardScaler()
 
# fitting
scalar.fit(X_train_norm_df)
X_train_stan = scalar.transform(X_train_norm_df)
X_train_stan_df = pd.DataFrame(X_train_stan)

scalar.fit(X_test_norm_df)
X_test_stan = scalar.transform(X_test_norm_df)
X_test_stan_df = pd.DataFrame(X_test_stan)

In [269]:
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_train_stan_df, y_train_c, test_size=0.30, random_state=42)

In [272]:
from sklearn.ensemble import VotingRegressor

models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X_train_3, X_test_3, y_train_3, y_test_3)
print(scores)
# create the ensemble
ensemble = VotingRegressor(estimators=models, weights=scores)
# fit the ensemble on the training dataset
ensemble.fit(X_train_stan_df, y_train_c)
# make predictions on test set
yhat_c = ensemble.predict(X_test_stan_df)

[0.0711975085875399, 0.07555883993661283, 0.09705616926554332]


## Predicting d

In [277]:
x_train_d_sensor1 = sensor1_X_train[['C','D','E','M','N','O','P']]

In [278]:
y_train_d = y_train['Property_D']

In [279]:
x_train_d_sensor2 = sensor2_X_train

In [280]:
x_train_d_together = pd.concat([x_train_d_sensor1, x_train_d_sensor2], axis=1)

In [281]:
x_test_d_sensor1 =  sensor1_X_test[['C','D','E','M','N','O','P']]

In [282]:
x_test_d_sensor2 = sensor2_X_test

In [283]:
x_test_d_together = pd.concat([x_test_d_sensor1, x_test_d_sensor2], axis=1)

In [284]:
# Normalize
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(x_train_d_together)

# transform training data
X_train_norm = norm.transform(x_train_d_together)
X_train_norm_df = pd.DataFrame(X_train_norm)

# transform testing dataabs
X_test_norm = norm.transform(x_test_d_together)
X_test_norm_df = pd.DataFrame(X_test_norm)

In [285]:
#  Standardize
from sklearn.preprocessing import StandardScaler
 
scalar = StandardScaler()
 
# fitting
scalar.fit(X_train_norm_df)
X_train_stan = scalar.transform(X_train_norm_df)
X_train_stan_df = pd.DataFrame(X_train_stan)

scalar.fit(X_test_norm_df)
X_test_stan = scalar.transform(X_test_norm_df)
X_test_stan_df = pd.DataFrame(X_test_stan)

In [286]:
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_train_stan_df, y_train_d, test_size=0.30, random_state=42)

In [289]:
from sklearn.ensemble import VotingRegressor

models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X_train_3, X_test_3, y_train_3, y_test_3)
print(scores)
# create the ensemble
ensemble = VotingRegressor(estimators=models, weights=scores)
# fit the ensemble on the training dataset
ensemble.fit(X_train_stan_df, y_train_d)
# make predictions on test set
yhat_d = ensemble.predict(X_test_stan_df)

[89.47406363915752, 66.7759448878805, 97.72427955395607]


## Predicting e

In [294]:
x_train_e_sensor1 = sensor1_X_train[['B','C','D','G','H','I','J','M','N','O','P']]

In [295]:
y_train_e = y_train['Property_E']

In [296]:
list_1 = sensor2_X_train.columns

In [297]:
list_4 = []

for i in list_1:
    if int(i[:4]) in range(350,450):
        list_4.append(i)

for i in list_1:
    if int(i[:4]) in range(800,1850):
        list_4.append(i)

In [298]:
x_train_e_sensor2 = sensor2_X_train[list_4]

In [299]:
x_train_e_together = pd.concat([x_train_e_sensor1, x_train_e_sensor2], axis=1)

In [300]:
x_test_e_sensor1 =  sensor1_X_test[['B','C','D','G','H','I','J','M','N','O','P']]

In [301]:
x_test_e_sensor2 = sensor2_X_test[list_4]

In [302]:
x_test_e_together = pd.concat([x_test_e_sensor1, x_test_e_sensor2], axis=1)

In [303]:
# Normalize
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(x_train_e_together)

# transform training data
X_train_norm = norm.transform(x_train_e_together)
X_train_norm_df = pd.DataFrame(X_train_norm)

# transform testing dataabs
X_test_norm = norm.transform(x_test_e_together)
X_test_norm_df = pd.DataFrame(X_test_norm)

In [304]:
#  Standardize
from sklearn.preprocessing import StandardScaler
 
scalar = StandardScaler()
 
# fitting
scalar.fit(X_train_norm_df)
X_train_stan = scalar.transform(X_train_norm_df)
X_train_stan_df = pd.DataFrame(X_train_stan)

scalar.fit(X_test_norm_df)
X_test_stan = scalar.transform(X_test_norm_df)
X_test_stan_df = pd.DataFrame(X_test_stan)

In [305]:
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_train_stan_df, y_train_e, test_size=0.30, random_state=42)

In [308]:
from sklearn.ensemble import VotingRegressor

models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X_train_3, X_test_3, y_train_3, y_test_3)
print(scores)
# create the ensemble
ensemble = VotingRegressor(estimators=models, weights=scores)
# fit the ensemble on the training dataset
ensemble.fit(X_train_stan_df, y_train_e)
# make predictions on test set
yhat_e = ensemble.predict(X_test_stan_df)

[86.21591354496412, 136.42489000145306, 93.56655344043632]


## Predicting f

In [314]:
x_train_f_sensor1 = sensor1_X_train

In [315]:
y_train_f = y_train['Property_F']

In [316]:
x_train_f_sensor2 = sensor2_X_train

In [317]:
x_train_f_together = pd.concat([x_train_f_sensor1, x_train_f_sensor2], axis=1)

In [318]:
x_test_f_sensor1 =  sensor1_X_test

In [319]:
x_test_f_sensor2 = sensor2_X_test

In [320]:
x_test_f_together = pd.concat([x_test_f_sensor1, x_test_f_sensor2], axis=1)

In [321]:
# Normalize
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(x_train_f_together)

# transform training data
X_train_norm = norm.transform(x_train_f_together)
X_train_norm_df = pd.DataFrame(X_train_norm)

# transform testing dataabs
X_test_norm = norm.transform(x_test_f_together)
X_test_norm_df = pd.DataFrame(X_test_norm)

In [322]:
#  Standardize
from sklearn.preprocessing import StandardScaler
 
scalar = StandardScaler()
 
# fitting
scalar.fit(X_train_norm_df)
X_train_stan = scalar.transform(X_train_norm_df)
X_train_stan_df = pd.DataFrame(X_train_stan)

scalar.fit(X_test_norm_df)
X_test_stan = scalar.transform(X_test_norm_df)
X_test_stan_df = pd.DataFrame(X_test_stan)

In [323]:
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_train_stan_df, y_train_f, test_size=0.30, random_state=42)

In [326]:
from sklearn.ensemble import VotingRegressor

models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X_train_3, X_test_3, y_train_3, y_test_3)
print(scores)
# create the ensemble
ensemble = VotingRegressor(estimators=models, weights=scores)
# fit the ensemble on the training dataset
ensemble.fit(X_train_stan_df, y_train_f)
# make predictions on test set
yhat_f = ensemble.predict(X_test_stan_df)

[19.481367732055507, 30.583875908862066, 22.549348865009982]


In [328]:
final_preds = yhat_a+yhat_b+yhat_c+yhat_e+yhat_d+yhat_f

In [330]:
temp_store_df = pd.DataFrame(final_preds)

In [331]:
test_df_id = test_df['Id']

In [332]:
submission = pd.concat([test_df_id, temp_store_df], axis=1)

In [333]:
submission.rename({0:"Predicted"},axis=1, inplace=True)

In [335]:
submission.to_csv('file12.csv', index=False)